<a href="https://colab.research.google.com/github/kaledai069/Crossword-Solver-A-Neural-Transformer-based-Approach/blob/master/Assessing_T5_small_model_for_its_scoring_ability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -q wordsegment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 17.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import time
import datetime
import random
import string
import re
import unicodedata

from pprint import pprint
from tqdm import tqdm
from transformers import AutoTokenizer, T5ForConditionalGeneration
from wordsegment import load, segment
load()

In [ ]:
crossword_path = "/content/gdrive/MyDrive/Clue-Answer Dataset/Clue-Answer-Dataset_Version_2.csv"
t5_reranker_path = "/content/gdrive/MyDrive/Second Pass Model/t5_small_new_dataset_2EPOCHS/"
t5_model_name = 't5-small'

byt5_reranker_path = "/content/gdrive/MyDrive/Second Pass Model/byt5_reranker/"
byt5_model_name = 'google/byt5-small'

In [ ]:
clue_answer_df = pd.read_csv(crossword_path)
clue_answer_df.head()

,Unnamed: 0,Clue,Answer,Segmented Answer,answer_length
0,0,cpa for example crossword clue,taxpro,tax pro,6.0
1,1,cordial crossword clue,warm,warm,4.0
2,2,joe without jolt crossword clue,decaf,decaf,5.0
3,3,bouillabaisse bit crossword clue,mussel,mussel,6.0
4,4,pre-gps travel aid crossword clue,aaamap,aaa map,6.0


In [ ]:
clue_answer_df['Unsegmented Answer'] = clue_answer_df['Segmented Answer'].apply(lambda x: str(x).replace(' ', ''))
all_answers = clue_answer_df['Unsegmented Answer'].unique().tolist()

In [ ]:
def get_random_clue_answer_pair(no_samples = 1, clue_answer_df = None):
    '''
    Should get me a random clue-answer pair from the dataset
    '''
    if clue_answer_df is None:
        raise ValueError("clue_answer_df must be provided.")

    random_indices = np.random.choice(clue_answer_df.index, size = no_samples, replace=False)
    random_sample = clue_answer_df.loc[random_indices]
    random_clues = random_sample['Clue'].tolist()
    random_answers = random_sample['Unsegmented Answer'].tolist()

    return zip(random_clues, random_answers)

In [ ]:
def get_randomly_edited_string(input_str, no_of_generation):
    '''
        Generate random edit-distance for 1 or 2 edit-distance randomly
        Input: input_str -> string; no_of_generation -> total generation in action
        Returns -> A list of 20-30 random strings with 1 or 2 edit distances.
    '''
    result = []

    i = 0
    while i < no_of_generation:
        choice = random.choice([1, 2])

        if choice == 1:
            # 1-edit distance
            edit_pos = random.randint(0, len(input_str) - 1)
            edit_char = random.choice(string.ascii_lowercase)
            new_str = input_str[:edit_pos] + edit_char + input_str[edit_pos + 1:]

        else:
            # 2-edit distances
            edit_pos1, edit_pos2 = random.sample(range(len(input_str)), 2)
            edit_char1, edit_char2 = random.choices(string.ascii_lowercase, k = 2)
            new_str = (
                input_str[:edit_pos1]
                + edit_char1
                + input_str[edit_pos1 + 1 : edit_pos2]
                + edit_char2
                + input_str[edit_pos2 + 1:]
            )

        i += 1
        if new_str[:len(input_str)] != input_str:
            result.append((new_str[:len(input_str)]))
        else:
            i -= 1

    return result

'''
    Plan out - lets takes randomly done edited 1 or 2 distance answers with equal string length
             - take 10-random answers from the dataset
'''
def get_random_answers_from_dataset(answer, all_answers, no_of_generation):
    output_random_answers = []
    range_of_random_indices = [random.randint(0, len(all_answers)) for _ in range(30)]

    i = 0
    while i < no_of_generation:
        random_index = random.randint(0, len(all_answers))

        if all_answers[random_index] != answer and len(all_answers[random_index]) == len(answer):
            output_random_answers.append(all_answers[random_index])
            i += 1

    return output_random_answers

In [ ]:
def get_negative_answers(answer, all_answers, no_randomly_edited, no_random_from_dataset):
    candidate_negatives = []

    randomly_edited_strings = get_randomly_edited_string(answer, no_randomly_edited)
    random_answers_from_dataset = get_random_answers_from_dataset('example', all_answers, no_random_from_dataset)

    candidate_negatives.extend(randomly_edited_strings)
    candidate_negatives.extend(random_answers_from_dataset)

    return candidate_negatives

In [ ]:
def preprocess_clue_fn(clue):
    clue = str(clue)
    clue = ''.join(c for c in unicodedata.normalize('NFD', clue) if unicodedata.category(c) != 'Mn')

    clue = re.sub("\x17|\x18|\x93|\x94|“|”|''|\"\"", "\"", clue)
    clue = re.sub("\x85|…", "...", clue)
    clue = re.sub("\x91|\x92|‘|’", "'", clue)

    clue = re.sub("‚", ",", clue)
    clue = re.sub("—|–", "-", clue)
    clue = re.sub("¢", " cents", clue)
    clue = re.sub("¿|¡|^;|\{|\}", "", clue)
    clue = re.sub("÷", "division", clue)
    clue = re.sub("°", " degrees", clue)

    euro = re.search("^£[0-9]+(,*[0-9]*){0,}| £[0-9]+(,*[0-9]*){0,}", clue)
    if euro:
        num = clue[:euro.end()]
        rest_clue = clue[euro.end():]
        clue = num + " Euros" + rest_clue
        clue = re.sub(", Euros", " Euros", clue)
        clue = re.sub("Euros [Mm]illion", "million Euros", clue)
        clue = re.sub("Euros [Bb]illion", "billion Euros", clue)
        clue = re.sub("Euros[Kk]", "K Euros", clue)
        clue = re.sub(" K Euros", "K Euros", clue)
        clue = re.sub("£", "", clue)

    clue = re.sub(" *\(\d{1,},*\)$| *\(\d{1,},* \d{1,}\)$", "", clue)

    clue = re.sub("&amp;", "&", clue)
    clue = re.sub("&lt;", "<", clue)
    clue = re.sub("&gt;", ">", clue)

    clue = re.sub("e\.g\.|for ex\.", "for example", clue)
    clue = re.sub(": [Aa]bbreviat\.|: [Aa]bbrev\.|: [Aa]bbrv\.|: [Aa]bbrv|: [Aa]bbr\.|: [Aa]bbr", " abbreviation", clue)
    clue = re.sub("abbr\.|abbrv\.", "abbreviation", clue)
    clue = re.sub("Abbr\.|Abbrv\.", "Abbreviation", clue)
    clue = re.sub("\(anag\.\)|\(anag\)", "(anagram)", clue)
    clue = re.sub("org\.", "organization", clue)
    clue = re.sub("Org\.", "Organization", clue)
    clue = re.sub("Grp\.|Gp\.", "Group", clue)
    clue = re.sub("grp\.|gp\.", "group", clue)
    clue = re.sub(": Sp\.", " (Spanish)", clue)
    clue = re.sub("\(Sp\.\)|Sp\.", "(Spanish)", clue)
    clue = re.sub("Ave\.", "Avenue", clue)
    clue = re.sub("Sch\.", "School", clue)
    clue = re.sub("sch\.", "school", clue)
    clue = re.sub("Agcy\.", "Agency", clue)
    clue = re.sub("agcy\.", "agency", clue)
    clue = re.sub("Co\.", "Company", clue)
    clue = re.sub("co\.", "company", clue)
    clue = re.sub("No\.", "Number", clue)
    clue = re.sub("no\.", "number", clue)
    clue = re.sub(": [Vv]ar\.", " variable", clue)
    clue = re.sub("Subj\.", "Subject", clue)
    clue = re.sub("subj\.", "subject", clue)
    clue = re.sub("Subjs\.", "Subjects", clue)
    clue = re.sub("subjs\.", "subjects", clue)

    theme_clue = re.search("^.+\|[A-Z]{1,}", clue)
    if theme_clue:
        clue = re.sub("\|", " | ", clue)

    if "Partner of" in clue:
        clue = re.sub("Partner of", "", clue)
        clue = clue + " and ___"

    link = re.search("^.+-.+ [Ll]ink$", clue)
    if link:
        no_link = re.search("^.+-.+ ", clue)
        x_y = clue[no_link.start():no_link.end() - 1]
        x_y_lst = x_y.split("-")
        clue = x_y_lst[0] + " ___ " + x_y_lst[1]

    follower = re.search("^.+ [Ff]ollower$", clue)
    if follower:
        no_follower = re.search("^.+ ", clue)
        x = clue[:no_follower.end() - 1]
        clue = x + " ___"

    preceder = re.search("^.+ [Pp]receder$", clue)
    if preceder:
        no_preceder = re.search("^.+ ", clue)
        x = clue[:no_preceder.end() - 1]
        clue = "___ " + x

    if re.search("--[^A-Za-z]|--$", clue):
        clue = re.sub("--", "__", clue)
    if not re.search("_-[A-Za-z]|_-$", clue):
        clue = re.sub("_-", "__", clue)

    clue = re.sub("_{2,}", "___", clue)

    clue = re.sub("\?$", " (wordplay)", clue)

    nonverbal = re.search("\[[^0-9]+,* *[^0-9]*\]", clue)
    if nonverbal:
        clue = re.sub("\[|\]", "", clue)
        clue = clue + " (nonverbal)"

    if clue[:4] == "\"\"\" " and clue[-4:] == " \"\"\"":
        clue = "\"" + clue[4:-4] + "\""
    if clue[:4] == "''' " and clue[-4:] == " '''":
        clue = "'" + clue[4:-4] + "'"
    if clue[:3] == "\"\"\"" and clue[-3:] == "\"\"\"":
        clue = "\"" + clue[3:-3] + "\""
    if clue[:3] == "'''" and clue[-3:] == "'''":
        clue = "'" + clue[3:-3] + "'"

    return clue

def post_process_clue(clue):
    clue = preprocess_clue_fn(clue)
    if clue[-3:] == '. .':
        clue = clue[:-3]
    elif clue[-3:] == ' ..':
        clue = clue[:-3]
    elif clue[-2:] == '..':
        clue = clue[:-2]
    elif clue[-1] == '.':
        clue = clue[:-1]
    return clue

In [ ]:
def get_scores(clue, answer_list, model, tokenizer):
    segmented_answer_list = []
    device = model.device

    for ans in answer_list:
        segmented_answer_list.append(" ".join(segment(ans.lower())))
    clue = post_process_clue(clue)
    scores = []

    with torch.no_grad(), torch.inference_mode():
        input = tokenizer(['Q: ' + clue], return_tensors = 'pt')['input_ids'].to(device)
        model.eval()

        for answer in segmented_answer_list:
            label = tokenizer([answer], return_tensors = 'pt')['input_ids'].to(device)
            loss = model(input, labels = label)
            answer_length = label.shape[1]
            score = loss[0].item() * answer_length
            scores.append(score)

    return scores

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained(t5_reranker_path)
t5_model.eval().to(device)

byt5_tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")
byt5_model = T5ForConditionalGeneration.from_pretrained(byt5_reranker_path)
byt5_model.eval().to(device)
print()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def get_test_dataset(test_size, CA_df, answers_list):
    size = test_size

    test_dataset = []
    clue_answer_pairs = get_random_clue_answer_pair(size, CA_df)

    for clue, answer in clue_answer_pairs:
        candidate_negatives = get_negative_answers(answer, answers_list, 75, 25)
        candidate_negatives.insert(0, answer)
        test_dataset.append([clue, candidate_negatives])
    return test_dataset

In [ ]:
def get_recall_k(ranks, k):
    count = 0
    for rank in ranks:
        if rank <= k:
            count += 1
    return count / len(ranks)

def mean_reciprocal_rank(ranks):
    return sum(1.0 / rank for rank in ranks) / len(ranks)

In [ ]:
test_dataset = get_test_dataset(500, clue_answer_df, all_answers)

In [ ]:
all_scores = []
for clue, answer_list in tqdm(test_dataset, ncols = 120):
    scores = []
    scores = get_scores(clue, answer_list, t5_model, t5_tokenizer)
    all_scores.append(scores)

ranks = []
for score_list in all_scores:
    ranks.append(score_list.index(min(score_list)) + 1)

for k in [1, 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
    print(f"Recall @ {k}: {get_recall_k(ranks, k)}")

100%|█████████████████████████████████████████████████████████████████████████████████| 500/500 [12:00<00:00,  1.44s/it]

Recall @ 1: 0.774
Recall @ 2: 0.774
Recall @ 5: 0.778
Recall @ 10: 0.79
Recall @ 20: 0.8
Recall @ 30: 0.808
Recall @ 40: 0.822
Recall @ 50: 0.828
Recall @ 60: 0.84
Recall @ 70: 0.846
Recall @ 80: 0.884
Recall @ 90: 0.946
Recall @ 100: 1.0


In [ ]:
mean_reciprocal_rank(ranks)

0.7800076603218346

In [ ]:
all_scores = []
for clue, answer_list in tqdm(test_dataset, ncols = 120):
    scores = []
    scores = get_scores(clue, answer_list, byt5_model, byt5_tokenizer)
    all_scores.append(scores)

ranks = []
for score_list in all_scores:
    ranks.append(score_list.index(min(score_list)) + 1)

for k in [1, 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
    print(f"Recall @ {k}: {get_recall_k(ranks, k)}")

100%|█████████████████████████████████████████████████████████████████████████████████| 500/500 [15:39<00:00,  1.88s/it]

Recall @ 1: 0.63
Recall @ 2: 0.632
Recall @ 5: 0.64
Recall @ 10: 0.652
Recall @ 20: 0.68
Recall @ 30: 0.694
Recall @ 40: 0.712
Recall @ 50: 0.722
Recall @ 60: 0.742
Recall @ 70: 0.762
Recall @ 80: 0.814
Recall @ 90: 0.914
Recall @ 100: 0.996


In [ ]:
mean_reciprocal_rank(ranks)

0.640894651759727